# Clean - Clean Dataset
This example illustrate how to perform meta-blocking on a clean-clean dataset (data linkage), so when we have two datasets that do not contain duplicates and we want to discover the duplicates between them. 

In [1]:
import sparker
import random
import pandas as pd

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [5]:
df = pd.read_json('/home/aditi/BigData/AAI_Project/dataset/df1_F1.json')

In [6]:
df

brand_left                                   description_left  \
0     hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...   
1              None                                               None   
2            asrock  intel h81 chipset x2 ddr3 x1 pci e x16 x1 pci ...   
3           fujitsu  description fujitsu 146gb ultra320 3 5 inchscs...   
4              benq  led backlit 1920x1080 resolution 12 000 000 1 ...   
...             ...                                                ...   
3753  hp enterprise  description 5 x 72 8gb ultra320 3 5 inch scsi ...   
3754  hp enterprise  description 2 x 72 8gb ultra320 3 5 inch scsi ...   
3755  hp enterprise  description 10 x 72 8gb ultra320 3 5 inch scsi...   
3756  hp enterprise  description 5 x 72 8gb ultra320 3 5 inch scsi ...   
3757           None                                               None   

          price_left                              specTableContent_left  \
0               None  specifications category proliant processor sub...   
1               None  categorie moederborden merk asus product asus ...   
2               None                                               None   
3     389 94 cad cad                                               None   
4               None                                               None   
...              ...                                                ...   
3753            None                                               None   
3754            None                                               None   
3755            None                                               None   
3756            None                                               None   
3757            None  quick specs buffer 256 mb storage capacity 250...   

                                             title_left  new_id  
0     495906 b21 hp x5560 2 80ghz ml350 g6 , null ne...       0  
1               asus prime x299 deluxe prijzen tweakers       1  
2     asrock h81m dgs r2 0 intel h81 socket 1150 ddr...       2  
3     null , mba3147np fujitsu 146 gb u320 scsi nhp 15k       3  
4     benq zowie rl2455 e sports 24 1920x1080 tn wid...       4  
...                                                 ...     ...  
3753  null , 306645 003 hp 72 8 gb u320 scsi 15k new...    3753  
3754  null , 306645 003 hp 72 8 gb u320 scsi 15k new...    3754  
3755  null , 306645 003 hp 72 8 gb u320 scsi 15k new...    3755  
3756  null , 306645 003 hp 72 8 gb u320 scsi 15k new...    3756  
3757  samsung 840 series 250gb 2 5in sata3 mdx solid...    3757  

[3758 rows x 6 columns]

## Load the data
sparkER provides wrappers to load CSV and JSON files.

First, we load the first dataset, and we extract the maximum id. The profiles ids of the second dataset will be assigned starting from this one.

*real_id_field* is the field that contains the identifier of the record.

In [13]:
# Profiles contained in the first dataset
profiles1 = sparker.JSONWrapper.load_profiles(file_path = '/Final_Project/sample/DF_op/Df1_F1.json',
                                             start_id_from = 0,
                                             real_id_field = "new_id",
                                            source_id = 0)
# Max profile id in the first dataset, used to separate the profiles in the next phases
separator_id = profiles1.map(lambda profile: profile.profile_id).max()
# Separators, used during blocking to understand from which dataset a profile belongs. It is an array because sparkER
# could work with multiple datasets
separator_ids = [separator_id]

In [14]:
type(separator_id)

int

In [15]:
separator_id

3757

In [16]:
profiles1.take(5)
profiles1.toDF().show(3698)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{brand_left, hp ...|          0|         0|        0|
|[{specTableConten...|          1|         1|        0|
|[{brand_left, asr...|          2|         2|        0|
|[{brand_left, fuj...|          3|         3|        0|
|[{brand_left, ben...|          4|         4|        0|
|[{brand_left, tp ...|          5|         5|        0|
|[{brand_left, tp ...|          6|         6|        0|
|[{brand_left, tp ...|          7|         7|        0|
|[{brand_left, tp ...|          8|         8|        0|
|[{brand_left, tp ...|          9|         9|        0|
|[{brand_left, asu...|         10|        10|        0|
|[{brand_left, asu...|         11|        11|        0|
|[{brand_left, log...|         12|        12|        0|
|[{brand_left, 216...|         13|        13|        0|
|[{brand_left, amd...|         14|        14|   

In [17]:
### by me
# col_p1 = ["attributes","original_id","profile_id","source_id"]

# ans_profiles1 = spark.createDataFrame(data = profiles1, schema = col_p1)
# lst = []
# data_p1 = ans_profiles1.collect()
# for row in data_p1:
#     print(f"{row[2]}")
    
#     lst.append(row[2])
#     #if(row[0]==)

In [18]:
# for i in lst:
#     print(i)

In [19]:
# len(lst)

In [20]:
a = profiles1.take(29)
a

[{'profile_id': 0, 'attributes': [{'key': 'brand_left', 'value': 'hp enterprise'}, {'key': 'description_left', 'value': 'description intel xeon x5560 ml350 g6 2 80ghz 4 core 8mb 95w full processor option kitpart number s option part 495906 b21'}, {'key': 'specTableContent_left', 'value': 'specifications category proliant processor sub category xeon generation g6 part number 495906 b21 products id 459573 product type processor upgrade processor type intel xeon processor core quad core processor qty 1 clock speed 2 8ghz bus speed 1333mhz l2 cache 1mb l3 cache 8mb 64 bit processing yes process technology 45nm processor socket socket b lga 1366 thermal design power 95w'}, {'key': 'title_left', 'value': '495906 b21 hp x5560 2 80ghz ml350 g6 , null new wholesale price'}], 'original_id': '0', 'source_id': 0},
 {'profile_id': 1, 'attributes': [{'key': 'specTableContent_left', 'value': 'categorie moederborden merk asus product asus prime x299 deluxe uitvoering asus prime x299 deluxe ean 4712900

Let's visualize a profile to check if they are correctly loaded

In [21]:
print(profiles1.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'brand_left', 'value': 'hp enterprise'}, {'key': 'description_left', 'value': 'description intel xeon x5560 ml350 g6 2 80ghz 4 core 8mb 95w full processor option kitpart number s option part 495906 b21'}, {'key': 'specTableContent_left', 'value': 'specifications category proliant processor sub category xeon generation g6 part number 495906 b21 products id 459573 product type processor upgrade processor type intel xeon processor core quad core processor qty 1 clock speed 2 8ghz bus speed 1333mhz l2 cache 1mb l3 cache 8mb 64 bit processing yes process technology 45nm processor socket socket b lga 1366 thermal design power 95w'}, {'key': 'title_left', 'value': '495906 b21 hp x5560 2 80ghz ml350 g6 , null new wholesale price'}], 'original_id': '0', 'source_id': 0}


Loads the second dataset and extract the max id (it will be used later)

In [23]:
profiles2 = sparker.JSONWrapper.load_profiles('/Final_Project/sample/DF_op/Df2_F2.json/', 
                                              start_id_from = separator_id+1, 
                                              real_id_field = "new_id",
                                            source_id = 1)
# Max profile id
max_profile_id = profiles2.map(lambda profile: profile.profile_id).max()

In [24]:
profiles2.count()

7220

In [25]:
profiles2.toDF().show(3226)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{brand_right, hp...|          0|      3758|        1|
|[{specTableConten...|          1|      3759|        1|
|[{specTableConten...|          2|      3760|        1|
|[{specTableConten...|          3|      3761|        1|
|[{specTableConten...|          4|      3762|        1|
|[{specTableConten...|          5|      3763|        1|
|[{brand_right, zo...|          6|      3764|        1|
|[{specTableConten...|          7|      3765|        1|
|[{specTableConten...|          8|      3766|        1|
|[{specTableConten...|          9|      3767|        1|
|[{description_rig...|         10|      3768|        1|
|[{brand_right, tp...|         11|      3769|        1|
|[{description_rig...|         12|      3770|        1|
|[{description_rig...|         13|      3771|        1|
|[{brand_right, 21...|         14|      3772|   

Finally, concatenate the two RDDs

In [26]:
profiles = profiles1.union(profiles2)

In [27]:
profiles.take(2)

[{'profile_id': 0, 'attributes': [{'key': 'brand_left', 'value': 'hp enterprise'}, {'key': 'description_left', 'value': 'description intel xeon x5560 ml350 g6 2 80ghz 4 core 8mb 95w full processor option kitpart number s option part 495906 b21'}, {'key': 'specTableContent_left', 'value': 'specifications category proliant processor sub category xeon generation g6 part number 495906 b21 products id 459573 product type processor upgrade processor type intel xeon processor core quad core processor qty 1 clock speed 2 8ghz bus speed 1333mhz l2 cache 1mb l3 cache 8mb 64 bit processing yes process technology 45nm processor socket socket b lga 1366 thermal design power 95w'}, {'key': 'title_left', 'value': '495906 b21 hp x5560 2 80ghz ml350 g6 , null new wholesale price'}], 'original_id': '0', 'source_id': 0},
 {'profile_id': 1, 'attributes': [{'key': 'specTableContent_left', 'value': 'categorie moederborden merk asus product asus prime x299 deluxe uitvoering asus prime x299 deluxe ean 4712900

In [28]:
profiles.count()

10978

In [29]:
profiles.toDF().show(4590) #b000jz4hqo

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{brand_left, hp ...|          0|         0|        0|
|[{specTableConten...|          1|         1|        0|
|[{brand_left, asr...|          2|         2|        0|
|[{brand_left, fuj...|          3|         3|        0|
|[{brand_left, ben...|          4|         4|        0|
|[{brand_left, tp ...|          5|         5|        0|
|[{brand_left, tp ...|          6|         6|        0|
|[{brand_left, tp ...|          7|         7|        0|
|[{brand_left, tp ...|          8|         8|        0|
|[{brand_left, tp ...|          9|         9|        0|
|[{brand_left, asu...|         10|        10|        0|
|[{brand_left, asu...|         11|        11|        0|
|[{brand_left, log...|         12|        12|        0|
|[{brand_left, 216...|         13|        13|        0|
|[{brand_left, amd...|         14|        14|   

In [30]:
newColumns = ["attributes","original_id","profile_id","source_id"]
p = spark.createDataFrame(data = profiles, schema = newColumns)

In [31]:
p.show()

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{brand_left, hp ...|          0|         0|        0|
|[{specTableConten...|          1|         1|        0|
|[{brand_left, asr...|          2|         2|        0|
|[{brand_left, fuj...|          3|         3|        0|
|[{brand_left, ben...|          4|         4|        0|
|[{brand_left, tp ...|          5|         5|        0|
|[{brand_left, tp ...|          6|         6|        0|
|[{brand_left, tp ...|          7|         7|        0|
|[{brand_left, tp ...|          8|         8|        0|
|[{brand_left, tp ...|          9|         9|        0|
|[{brand_left, asu...|         10|        10|        0|
|[{brand_left, asu...|         11|        11|        0|
|[{brand_left, log...|         12|        12|        0|
|[{brand_left, 216...|         13|        13|        0|
|[{brand_left, amd...|         14|        14|   

In [32]:
tokens_per_profile = profiles.map(lambda profile: (
            profile.attributes))

In [33]:
tokens_per_profile.take(4)

[[{'key': 'brand_left', 'value': 'hp enterprise'},
  {'key': 'description_left', 'value': 'description intel xeon x5560 ml350 g6 2 80ghz 4 core 8mb 95w full processor option kitpart number s option part 495906 b21'},
  {'key': 'specTableContent_left', 'value': 'specifications category proliant processor sub category xeon generation g6 part number 495906 b21 products id 459573 product type processor upgrade processor type intel xeon processor core quad core processor qty 1 clock speed 2 8ghz bus speed 1333mhz l2 cache 1mb l3 cache 8mb 64 bit processing yes process technology 45nm processor socket socket b lga 1366 thermal design power 95w'},
  {'key': 'title_left', 'value': '495906 b21 hp x5560 2 80ghz ml350 g6 , null new wholesale price'}],
 [{'key': 'specTableContent_left', 'value': 'categorie moederborden merk asus product asus prime x299 deluxe uitvoering asus prime x299 deluxe ean 4712900763881 sku 90mb0ty0 m0eay0 prime x299 deluxe tweakers id 776903 productprijs 398 95 pakketpost 

### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [34]:
# # Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# # respectively the id of profiles of the first dataset and the id of profiles of the second dataset
# gt = sparker.CSVWrapper.load_groundtruth('/Final_Project/sample/ground_truth_sample.csv',
#                                         'id1', 'id2')

In [35]:
# gt.toDF().show(1400)

In [36]:
# Converts the groundtruth by replacing original IDs with those given by Spark
# new_gt = sparker.Converters.convert_groundtruth(gt, profiles1, profiles2)

In [37]:
# new_gt

In [38]:
# len(new_gt)

In [39]:
# We can explore some pairs
# random.sample(new_gt, 5)

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [40]:
blocks = sparker.Blocking.create_blocks(profiles, separator_ids, 
                                        blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
                                        ngram_size=4)
print("Number of blocks",blocks.count())

Number of blocks 36069


In [41]:
sparker.BlockingKeysStrategies.ngrams_blocking

<function sparker.blocking_strategies.BlockingKeysStrategies.ngrams_blocking(attributes, keys_to_exclude=None, ngram_size=3)>

In [42]:
blocks.take(500)

[{'block_id': 0, 'profiles': [{0, 1, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 29, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 69, 70, 71, 72, 73, 77, 78, 79, 80, 81, 82, 83, 84, 85, 88, 89, 90, 91, 92, 93, 94, 95, 96, 99, 100, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 120, 122, 125, 126, 135, 136, 138, 140, 141, 142, 143, 145, 146, 148, 149, 150, 151, 154, 155, 156, 158, 159, 160, 161, 162, 164, 166, 167, 168, 169, 170, 173, 174, 176, 177, 179, 180, 182, 185, 188, 189, 190, 191, 192, 193, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 210, 211, 212, 213, 216, 219, 221, 222, 223, 224, 229, 230, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 246, 247, 251, 253, 254, 256, 257, 258, 259, 260, 261, 263, 264, 265, 266, 267, 268, 269, 270, 271, 276, 277, 281, 282, 284, 286, 288, 289, 290, 291, 293, 295, 296, 297, 298, 299, 300, 301, 302,

Let's continue by using the standard token blocking

In [43]:
blocks = sparker.Blocking.create_blocks(profiles, separator_ids)
print("Number of blocks",blocks.count())

Number of blocks 17563


In [44]:
blocks.take(500)

[{'block_id': 0, 'profiles': [{0, 2553, 2554}, {8786, 8779, 3758}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 1, 'profiles': [{0, 15, 16, 17, 55, 58, 59, 74, 75, 76, 100, 101, 122, 123, 124, 127, 128, 137, 138, 143, 145, 146, 147, 175, 178, 197, 198, 202, 221, 222, 223, 224, 233, 234, 235, 236, 243, 244, 245, 246, 247, 248, 249, 297, 298, 348, 350, 370, 371, 372, 382, 387, 405, 406, 407, 409, 414, 419, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 485, 494, 2553, 2554, 511, 2560, 2564, 2565, 2581, 2583, 541, 542, 570, 571, 573, 2624, 584, 591, 592, 594, 595, 596, 605, 606, 2656, 2657, 613, 2662, 2663, 2666, 2667, 2668, 2669, 2670, 2675, 2676, 2677, 2678, 2679, 2680, 2681, 2682, 2683, 2690, 644, 650, 651, 652, 653, 654, 655, 656, 677, 678, 679, 680, 681, 682, 683, 2729, 2730, 2731, 2735, 2732, 689, 2737, 2733, 2734, 2736, 2742, 2743, 2744, 2745, 708, 710, 711, 712, 2759, 2760, 2770, 2771, 724, 725, 726, 2774, 2776, 2777, 2778, 2779, 2780, 733, 734, 735, 7

In [45]:
blocks.take(1)

[{'block_id': 0, 'profiles': [{0, 2553, 2554}, {8786, 8779, 3758}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''}]

## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [46]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

In [47]:
blocks_purged.take(500)

[{'block_id': 0, 'profiles': [{0, 2553, 2554}, {8786, 8779, 3758}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 1, 'profiles': [{0, 15, 16, 17, 55, 58, 59, 74, 75, 76, 100, 101, 122, 123, 124, 127, 128, 137, 138, 143, 145, 146, 147, 175, 178, 197, 198, 202, 221, 222, 223, 224, 233, 234, 235, 236, 243, 244, 245, 246, 247, 248, 249, 297, 298, 348, 350, 370, 371, 372, 382, 387, 405, 406, 407, 409, 414, 419, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 485, 494, 2553, 2554, 511, 2560, 2564, 2565, 2581, 2583, 541, 542, 570, 571, 573, 2624, 584, 591, 592, 594, 595, 596, 605, 606, 2656, 2657, 613, 2662, 2663, 2666, 2667, 2668, 2669, 2670, 2675, 2676, 2677, 2678, 2679, 2680, 2681, 2682, 2683, 2690, 644, 650, 651, 652, 653, 654, 655, 656, 677, 678, 679, 680, 681, 682, 683, 2729, 2730, 2731, 2735, 2732, 689, 2737, 2733, 2734, 2736, 2742, 2743, 2744, 2745, 708, 710, 711, 712, 2759, 2760, 2770, 2771, 724, 725, 726, 2774, 2776, 2777, 2778, 2779, 2780, 733, 734, 735, 7

In [48]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8, separator_ids)

In [49]:
blocks_purged.take(500)

[{'block_id': 0, 'profiles': [{0, 2553, 2554}, {8786, 8779, 3758}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 1, 'profiles': [{0, 15, 16, 17, 55, 58, 59, 74, 75, 76, 100, 101, 122, 123, 124, 127, 128, 137, 138, 143, 145, 146, 147, 175, 178, 197, 198, 202, 221, 222, 223, 224, 233, 234, 235, 236, 243, 244, 245, 246, 247, 248, 249, 297, 298, 348, 350, 370, 371, 372, 382, 387, 405, 406, 407, 409, 414, 419, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 485, 494, 2553, 2554, 511, 2560, 2564, 2565, 2581, 2583, 541, 542, 570, 571, 573, 2624, 584, 591, 592, 594, 595, 596, 605, 606, 2656, 2657, 613, 2662, 2663, 2666, 2667, 2668, 2669, 2670, 2675, 2676, 2677, 2678, 2679, 2680, 2681, 2682, 2683, 2690, 644, 650, 651, 652, 653, 654, 655, 656, 677, 678, 679, 680, 681, 682, 683, 2729, 2730, 2731, 2735, 2732, 689, 2737, 2733, 2734, 2736, 2742, 2743, 2744, 2745, 708, 710, 711, 712, 2759, 2760, 2770, 2771, 724, 725, 726, 2774, 2776, 2777, 2778, 2779, 2780, 733, 734, 735, 7

In [50]:
print("Number of blocks_purged",blocks_purged.count())

Number of blocks_purged 17557


If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [51]:
# recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt, separator_ids)

# print("Recall", recall)
# print("Precision", precision)
# print("Number of comparisons", cmp_n)

## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [52]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [53]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
# gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [54]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 4172755


The number of edges
The number of matches (only if the groundtruth is provided)
The retained edges

In [55]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show()

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|       1832|            0|[{0, 8786, 46}, {...|
|        632|            0|[{128, 9730, 4}, ...|
|       1589|            0|[{224, 5120, 18},...|
|       1795|            0|[{2560, 4097, 8},...|
|        720|            0|[{2624, 9856, 26}...|
|       1283|            0|[{2656, 8195, 2},...|
|       2014|            0|[{736, 7169, 3}, ...|
|        693|            0|[{2816, 4043, 82}...|
|        857|            0|[{2880, 4923, 14}...|
|       2428|            0|[{3040, 10241, 22...|
|        884|            0|[{3136, 10320, 16...|
|       1668|            0|[{3424, 10241, 7}...|
|       1256|            0|[{3456, 7945, 16}...|
|       1078|            0|[{1632, 9231, 4},...|
|          0|            0|                  []|
|          0|            0|                  []|
|          0|            0|                  []|
|          0|       

### Reciprocal Weighted Node Pruning

In [56]:
lst1newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(truncate=False)

+-----------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [57]:
edges = results.flatMap(lambda x: x[2])

edges.count()

4172755

In [58]:
newColumns = ["profile_id","neighbor_id","neighbor_weight"]
ans = edges.toDF(newColumns)
# ans.sort_values(by=['neighbor_weight'])    #show(50000)

In [59]:

ans = spark.createDataFrame(data = edges, schema = newColumns)

In [60]:
ans.show(50000)

+----------+-----------+---------------+
|profile_id|neighbor_id|neighbor_weight|
+----------+-----------+---------------+
|         0|       8741|             35|
|         0|      10291|             34|
|         0|      10292|             31|
|         0|       8246|              9|
|         0|       8760|              8|
|         0|       8250|              7|
|         0|      10310|             34|
|         0|       8779|             49|
|         0|      10316|             34|
|         0|       8780|              9|
|         0|      10318|             35|
|         0|       8782|              9|
|         0|       8785|              9|
|         0|       8786|             46|
|         0|      10324|              8|
|         0|       9311|             37|
|         0|       9315|             32|
|         0|       9320|             33|
|         0|       4719|              8|
|         0|      10669|             35|
|         0|       7801|              7|
|         0|    

In [61]:
# ans.groupBy("profile_id").max("neighbor_weight").show()

In [62]:
max_by_group = (edges
  .map(lambda x: (x[0], x))  
   .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1]))
   .values())
                
# Convmax_by_group.collect()ert to PairwiseRD
  # Take maximum of the passed arguments by the last element (key)
  # equivalent to:
  # lambda x, y: x if x[-1] > y[-1] else y

In [63]:
max_by_group.take(60)

[(0, 8779, 49),
 (128, 8540, 12),
 (224, 3978, 51),
 (2560, 7987, 44),
 (2624, 8241, 26),
 (2656, 3843, 30),
 (736, 9527, 73),
 (2816, 4043, 82),
 (2880, 7604, 43),
 (3040, 10256, 27),
 (3136, 7950, 26),
 (3424, 8471, 46),
 (3456, 8032, 58),
 (1632, 10328, 42),
 (1536, 5471, 70),
 (2976, 8771, 32),
 (2688, 10505, 31),
 (3232, 8754, 82),
 (1344, 8904, 38),
 (3520, 8477, 30),
 (3552, 8944, 20),
 (3616, 8307, 43),
 (2048, 5973, 192),
 (2080, 6005, 100),
 (2112, 6047, 60),
 (96, 3838, 26),
 (2304, 6632, 90),
 (2496, 7129, 33),
 (2592, 3783, 19),
 (768, 9428, 33),
 (832, 4740, 72),
 (3008, 4305, 31),
 (1184, 9007, 42),
 (3264, 10548, 19),
 (3360, 9820, 45),
 (1408, 5316, 58),
 (3488, 9927, 62),
 (1472, 5404, 32),
 (3744, 8991, 50),
 (1792, 10624, 53),
 (1920, 5845, 66),
 (512, 7738, 35),
 (256, 8342, 50),
 (64, 4827, 12),
 (2336, 6657, 61),
 (3392, 4829, 29),
 (3712, 10419, 95),
 (1600, 5553, 70),
 (2752, 4422, 43),
 (3296, 4692, 34),
 (32, 3786, 31),
 (192, 7635, 46),
 (608, 10773, 75),
 (

In [64]:
edges.take(10)

[(0, 4112, 31),
 (0, 4113, 5),
 (0, 4114, 5),
 (0, 4116, 5),
 (0, 4119, 5),
 (0, 4124, 5),
 (0, 10277, 33),
 (0, 10278, 36),
 (0, 10286, 36),
 (0, 8240, 4)]

In [65]:
# lst1 = []
# dataCollect = ans.collect()
# for row in dataCollect:
#     lst1.append(row[0])
    
# list_set = set(lst1)
# s = list(list_set)
# s

In [66]:
# lst0 = []
# p_id = []
# n_id = []
# e = []
# d = {}

# def cal():
#     for i in s:
#         maxm = 1
#         for row in dataCollect:
#             if(i == row[0]):
#                 if(row[2] > maxm):
#                     maxm = row[2]
#                     #print(f"{row[0]}, {row[1]}, {row[2]}")
#                     p_id.append(row[0])
#                     n_id.append(row[1]-(separator_id+1))
#                     e.append(row[2])
#                     d = {"P_id": p_id, "N_id": n_id, "weight": e}
#     return(d)

# maxm = 0
# dataCollect = ans.collect()

# a = cal()
# df =  pd.DataFrame.from_dict(a)
# df

#             #lst1.append(row[0])

In [67]:
dataCollect = max_by_group.collect()
df =  pd.DataFrame(dataCollect)
df

0      1   2
0        0   8779  49
1      128   8540  12
2      224   3978  51
3     2560   7987  44
4     2624   8241  26
...    ...    ...  ..
3753  2239   6170   9
3754   287   4046  11
3755    31  10385  33
3756   159   3904  10
3757   255   8343   7

[3758 rows x 3 columns]

In [68]:
df.rename(columns = {0:'P_id', 1:'N_id',
                              2:'weight'}, inplace = True)
df.head()

P_id  N_id  weight
0     0  8779      49
1   128  8540      12
2   224  3978      51
3  2560  7987      44
4  2624  8241      26

In [69]:
n = df.filter(["P_id","N_id","weight"])
n = pd.DataFrame(n)
n

P_id   N_id  weight
0        0   8779      49
1      128   8540      12
2      224   3978      51
3     2560   7987      44
4     2624   8241      26
...    ...    ...     ...
3753  2239   6170       9
3754   287   4046      11
3755    31  10385      33
3756   159   3904      10
3757   255   8343       7

[3758 rows x 3 columns]

In [70]:
n["N_id"] = n["N_id"]-(separator_id+1)
new_df = n[(n['weight']>50)]
new_df

P_id  N_id  weight
2      224   220      51
6      736  5769      73
7     2816   285      82
12    3456  4274      58
14    1536  1713      70
...    ...   ...     ...
3741  3199   751      56
3742   671   759      54
3745  1407  5601      57
3747  1855  6509      62
3750  2687  6022      62

[1428 rows x 3 columns]

In [71]:
col = ["P_id","N_id","weight"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show()

+----+----+------+
|P_id|N_id|weight|
+----+----+------+
| 224| 220|    51|
| 736|5769|    73|
|2816| 285|    82|
|3456|4274|    58|
|1536|1713|    70|
|3232|4996|    82|
|2048|2215|   192|
|2080|2247|   100|
|2112|2289|    60|
|2304|2874|    90|
| 832| 982|    72|
|1408|1558|    58|
|3488|6169|    62|
|1792|6866|    53|
|1920|2087|    66|
|2336|2899|    61|
|3712|6661|    95|
|1600|1795|    70|
| 608|7015|    75|
|1760|4205|    63|
+----+----+------+
only showing top 20 rows



In [72]:
ss = SparkSession(sc)


In [73]:
df1 = ss.read.json('hdfs://localhost:9000/Final_Project/sample/DF_op/Df1_F1.json')
df1.show(5)

+----------+--------------------+------+----------+---------------------+--------------------+
|brand_left|    description_left|new_id|price_left|specTableContent_left|          title_left|
+----------+--------------------+------+----------+---------------------+--------------------+
|     canon|canon 2400 digita...|  2106|      null|                 null|canon 2400 digita...|
|     canon|canon 2400 digita...|  2107|      null|                 null|canon 2400 digita...|
| transcend|5 pack transcend ...|  2108|      null|                 null|5 pack transcend ...|
| transcend|5 pack transcend ...|  2109|      null|                 null|5 pack transcend ...|
| transcend|as high tech gadg...|  2110|      null|                 null|transcend 32gb sd...|
+----------+--------------------+------+----------+---------------------+--------------------+
only showing top 5 rows



In [74]:
df2 = ss.read.json('hdfs://localhost:9000/Final_Project/sample/DF_op/Df2_F2.json')
df2.show(5)

+-----------+--------------------+------+-----------+----------------------+--------------------+
|brand_right|   description_right|new_id|price_right|specTableContent_right|         title_right|
+-----------+--------------------+------+-----------+----------------------+--------------------+
|   fujifilm|the new compact i...|  2706|       null|                  null|fujifilm instax m...|
|   fujifilm|the new compact i...|  2707|       null|                  null|fujifilm instax m...|
|       null|netgear vma1100 1...|  2708|       null|  mfg model netgear...|netgear vma1100 1...|
|      canon|canon imageprogra...|  2709|       null|                  null|genuine canon ima...|
|      canon|canon imageprogra...|  2710|       null|                  null|genuine canon ima...|
+-----------+--------------------+------+-----------+----------------------+--------------------+
only showing top 5 rows



In [89]:
df2 = df2.withColumnRenamed("new_id" , "id_2")
df2.show(5)

+-----------+--------------------+----+-----------+----------------------+--------------------+
|brand_right|   description_right|id_2|price_right|specTableContent_right|         title_right|
+-----------+--------------------+----+-----------+----------------------+--------------------+
|   fujifilm|the new compact i...|2706|       null|                  null|fujifilm instax m...|
|   fujifilm|the new compact i...|2707|       null|                  null|fujifilm instax m...|
|       null|netgear vma1100 1...|2708|       null|  mfg model netgear...|netgear vma1100 1...|
|      canon|canon imageprogra...|2709|       null|                  null|genuine canon ima...|
|      canon|canon imageprogra...|2710|       null|                  null|genuine canon ima...|
+-----------+--------------------+----+-----------+----------------------+--------------------+
only showing top 5 rows



In [90]:
#common_df = df2.join(df_weight,df2.id_2 == df_weight.N_id,"inner")

In [91]:
# inner join on two dataframes
common_df = df1.join(df_weight,df1.new_id == df_weight.P_id,"inner")

In [92]:
common_df

DataFrame[brand_left: string, description_left: string, new_id: bigint, price_left: string, specTableContent_left: string, title_left: string, P_id: bigint, N_id: bigint, weight: bigint]

In [93]:
pd_common_df = common_df.toPandas()
pd_common_df

brand_left                                   description_left  \
0             seagate  description seagate constellation es 3 1 tb ho...   
1     western digital  wd red is the only hard drive designed for nas...   
2                None  incroyablement mince et l ger le hp elitebook ...   
3               intel  six core technology 3 5ghz clock speed 8 25mb ...   
4                None                                               None   
...               ...                                                ...   
1423    hp enterprise  description 2 x 146gb 2 5 inch serial attached...   
1424    hp enterprise  description 146gb 2 5 inch serial attached scs...   
1425    hp enterprise  description 5 x 300gb 2 5 inch serial attached...   
1426    hp enterprise  description storageworks 146 8gb 10k fibre cha...   
1427             None  plug and play for high performance with intel ...   

      new_id   price_left                              specTableContent_left  \
0        224   usd 149 95  specifications category seagate harddrive sub ...   
1        736         None  item weight 1 66 pounds item size 4 x 1 x 1 in...   
2       2816  cad 1925 00                                               None   
3       3456         None  processor number i7 7800x number of cores 6 nu...   
4       1536         None  categorie processors merk intel productserie c...   
...      ...          ...                                                ...   
1423    3199         None  specifications category proliant harddrive sub...   
1424     671         None  specifications category proliant harddrive sub...   
1425    1407         None  specifications category proliant harddrive sub...   
1426    1855         None  specifications category storageworks hdd sub c...   
1427    2687        49 87                                               None   

                                             title_left  P_id  N_id  weight  
0     null , st1000nm0033 seagate 1 tb 7 2k 3 5 6g s...   224   220      51  
1     wd red 6tb nas hard disk drive 5400 rpm class ...   736  5769      73  
2     ordinateur portable hp elitebook 850 g3 energy...  2816   285      82  
3     7th generation intel core i7 7800x 3 5ghz sock...  3456  4274      58  
4             intel core i5 6400 boxed prijzen tweakers  1536  1713      70  
...                                                 ...   ...   ...     ...  
1423  eg0146fartr hp 146 gb 6g 10k 2 5 dp sas , null...  3199   751      56  
1424  eg0146fartr hp 146 gb 6g 10k 2 5 dp sas hdd , ...   671   759      54  
1425  dg0300farvv hp 300 gb 6g 10k 2 5 dp sas , null...  1407  5601      57  
1426  bd1465822c hp 146 8 gb 10k fc al hdd , null ne...  1855  6509      62  
1427  hyperx fury ddr4 4 gb dimm 288 pin hx426c15fb ...  2687  6022      62  

[1428 rows x 9 columns]

In [99]:
common_df_2 = common_df.join(df2,common_df.N_id == df2.id_2,"inner")
pd_common_df_2 = common_df_2.toPandas()
# pd_common_df_2 = pd_common_df_2.drop(["Location","Location_2","P_id","N_id"], axis=1)
# pd_common_df_2

In [109]:
pd_common_df_2.iloc[2:12]

brand_left                                   description_left  new_id  \
2            None  incroyablement mince et l ger le hp elitebook ...    2816   
3           intel  six core technology 3 5ghz clock speed 8 25mb ...    3456   
4            None                                               None    1536   
5            None                                               None    3232   
6        logitech  long product name logitech c920 1920 x 1080pix...    2048   
7           kodak  langer produktname kodak power flash single us...    2080   
8            None  the dcs 6010l 2 megapixel panoramic wireless c...    2112   
9        kingston  move to faster performance with kingston s sdh...    2304   
10  hp enterprise  description 10 x 400gb serial attached scsi sa...     832   
11  hp enterprise  description 10 x 300gb 2 5 inch serial attache...    1408   

     price_left                              specTableContent_left  \
2   cad 1925 00                                               None   
3          None  processor number i7 7800x number of cores 6 nu...   
4          None  categorie processors merk intel productserie c...   
5          None  categorie processors merk intel productserie c...   
6          None                                               None   
7          None                                               None   
8          None  versie beschrijving datum 1 15 03 release note...   
9          None                                               None   
10         None  specifications category proliant harddrive sub...   
11         None  specifications category proliant harddrive sub...   

                                           title_left  P_id  N_id  weight  \
2   ordinateur portable hp elitebook 850 g3 energy...  2816   285      82   
3   7th generation intel core i7 7800x 3 5ghz sock...  3456  4274      58   
4           intel core i5 6400 boxed prijzen tweakers  1536  1713      70   
5     intel core i7 7700 boxed specificaties tweakers  3232  4996      82   
6   logitech c920 1920 x 1080pixels usb 2 0 black ...  2048  2215     192   
7   kodak power flash single use camera compact fi...  2080  2247     100   
8   2 megapixel panoramic wireless cloud camera dc...  2112  2289      60   
9   kingston digital 128gb sdxc uhs i speed class ...  2304  2874      90   
10  456896 001 hp 400 gb 10k 3 5 dp sas , null new...   832   982      72   
11  dg0300farvv hp 300 gb 6g 10k 2 5 dp sas , null...  1408  1558      58   

      brand_right                                  description_right  id_2  \
2            None  incroyablement mince et l ger le hp elitebook ...   285   
3           intel  six core technology 3 5ghz clock speed 8 25mb ...  4274   
4            None                                               None  1713   
5            None                                               None  4996   
6        logitech  long product name logitech c920 1920 x 1080pix...  2215   
7           kodak  langer produktname kodak power flash single us...  2247   
8            None  the dcs 6010l 2 megapixel panoramic wireless c...  2289   
9            None  move to faster performance with kingston s sdh...  2874   
10  hp enterprise  description 10 x 400gb serial attached scsi sa...   982   
11  hp enterprise  description 10 x 300gb 2 5 inch serial attache...  1558   

    price_right                             specTableContent_right  \
2   cad 1925 00                                               None   
3          None  processor number i7 7800x number of cores 6 nu...   
4          None  categorie processors merk intel productserie c...   
5          None  categorie processors merk intel productserie c...   
6          None                                               None   
7          None                                               None   
8          None  version description date 1 15 03 release notes...   
9         27 99                                               None   
10         None  s

In [103]:
#pd_common_df_2.to_json('/home/aditi/BigData/AAI_Project/dataset/Output.csv', header=True, index=False)

In [105]:
baseDir = 'hdfs://localhost:9000/Final_Project/sample/DF_op/Output.json'
common_df_2.write.option("header", True).mode('overwrite').json(baseDir)

########################################################################
########################################################################